# Evaluating Rerankers

In this notebook we will showcase how you can evaluate reranking models with Baguetter.

We showcase here the usage of our inference engine [Ofen](https://github/mixedbread-ai/ofen) for generating the embeddings.

In [ ]:
%pip install "ofen[torch]==0.0.1"

In [2]:
from ofen.models import CrossEncoder
from baguetter.evaluation import evaluate_retrievers, HFDataset
from baguetter.utils.model_helpers import create_post_processing_fn
from baguetter.indices import SearchEngine

## 1. Initialize the Reranker

In [3]:
reranker = CrossEncoder.from_pretrained("mixedbread-ai/mxbai-rerank-xsmall-v1")

# Convert model to half precision (FP16) for efficiency
reranker.half()

# Create the reranking function to rerank the search results
rerank_fn = create_post_processing_fn(reranker, batch_size=256)

## 2. Evaluate the Reranker on top of BMX

In [4]:
result = evaluate_retrievers(
    datasets=[HFDataset("mteb/scidocs")],
    retriever_factories={
        "engine": lambda: SearchEngine(
            post_process_fn=rerank_fn
        )
    } 
)

result.save("eval_results")

Evaluating  1 retrievers...
---------------------------------------------------------------
Datasets:  ['mteb/scidocs']
Top K:  100
Metrics:  ['ndcg@1', 'ndcg@5', 'ndcg@10', 'precision@1', 'precision@5', 'precision@10', 'mrr@1', 'mrr@5', 'mrr@10']
Ignore identical IDs:  True

Evaluating Dataset: mteb/scidocs
---------------------------------------------------------------
Starting Adding 25657 documents to engine...


Building inverted index: 100%|██████████| 61627/61627 [00:09<00:00, 6276.60it/s]


Adding 25657 documents to engine took 24.99 seconds
Starting Searching 1000 queries with engine...


Ranking: 100%|██████████| 782/782 [01:32<00:00,  8.48it/s]


Searching 1000 queries with engine took 98.81 seconds

Report (rounded):
---------------------------------------------------------------
#    Model      NDCG@1    NDCG@5    NDCG@10    P@1    P@5    P@10    MRR@1    MRR@5    MRR@10
---  -------  --------  --------  ---------  -----  -----  ------  -------  -------  --------
a    engine      0.206     0.143      0.171  0.206  0.126   0.088    0.206    0.287       0.3
